# Phase II: Data Curation, Exploratory Analysis and Plotting (5\%)

### Team Members:
- Deepanshu Bhagria 
- Ishaan Tayal
- Ignacio Torner Toledo
- Dev Patel


## Part 1: 
(1%) Expresses the central motivation of the project and explains the (at least) two key questions to be explored. Gives a summary of the data processing pipeline so a technical expert can easily follow along.

## (A) Problem Motivation 

The stock market changes every day, and investors often try to predict whether prices will rise or fall. While stock prices are influenced by many factors, we believe that historical trading data still contains valuable patterns that can help make predictions. In this project, we will use Apple’s (AAPL) historical stock data to explore whether features such as daily percent change, trading volume, and moving averages can help forecast the next day’s closing price.

### The key questions we aim to answer are:

##### 1) Which historical features (e.g., percent change, volume, moving averages) are most predictive of tomorrow’s closing price?

##### 2) How accurately can we predict the next day’s closing price using historical data?

### Motivating sources:

Yahoo Finance. "AAPL Historical Data." Yahoo Finance, https://finance.yahoo.com/quote/AAPL/history
. Accessed 26 Oct. 2025.

Investopedia. "Technical Analysis of Stocks." Investopedia, https://www.investopedia.com/terms/t/technicalanalysis.asp
. Accessed 26 Oct. 2025.

## (B) Summary of the Data Processing Pipeline

1) Collect data: Retrieve historical stock data using the Yahoo Finance API (yfinance).

2) Clean the data: Handle missing values, filter relevant columns, and compute derived features like moving averages and daily percent changes.

3) Visualize trends: Use plotting libraries like Matplotlib, Seaborn, and Plotly to explore patterns and relationships in the data.

4) Prepare for modeling: Select relevant features and split the data into training and testing sets for predictive modeling.

To process the data, we will first acquire Apple’s historical stock data using the yfinance API. After collection, we will clean the data by removing missing or inconsistent values, computing features like daily percent change and moving averages, and converting dates to a usable format. Next, we will visualize trends in the stock data, such as price changes over time, correlations with trading volume, and other patterns that may inform predictions. Finally, the cleaned data will be prepared for predictive modeling, where we will train and test models to forecast next-day closing prices. This workflow will help us answer our key questions and assess the potential of historical stock features in predicting market movements.

## Part 2: 
(2\%) Obtains, cleans, and merges all data sources involved in the project.

In [ ]:
# Importing Necessary Libraries
import yfinance as yf
import pandas as pd

#Loading the pre-processes data
df = yf.download('AAPL', start='2015-01-01', end='2025-10-25')
df.head()

/var/folders/v4/rl51r6ks60z5gjpkw48kqr7h0000gn/T/ipykernel_79209/197221887.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('AAPL', start='2015-01-01', end='2025-10-25')
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2015-01-02,24.261047,24.729270,23.821672,24.718174,212818400
2015-01-05,23.577578,24.110154,23.391177,24.030267,257142000
2015-01-06,23.579798,23.839428,23.218089,23.641931,263188400
2015-01-07,23.910439,24.010296,23.677436,23.788389,160423600
2015-01-08,24.829126,24.886823,24.121244,24.238856,237458000


In [25]:

def clean_stock_data(df):
    """
    Cleans stock data from yfinance:
    - Flattens MultiIndex columns
    - Resets index to make 'Date' a column
    - Drops missing values
    - Adds daily percent change and moving averages (5-day and 20-day)
    """
    
    # makes a copy of df
    df = df.copy()
    
    # Flatten MultiIndex columns
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [col[0] if col[1] else col[0] for col in df.columns]
    
    # Reset index so Date is a column
    df.reset_index(inplace=True)
    
    # Drop any rows with missing values
    df.dropna(subset=['Close', 'Volume'], inplace=True)
    
    # Convert Date to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Derived features
    df['Daily_Change'] = df['Close'].pct_change() * 100
    df['MA_5'] = df['Close'].rolling(5).mean()
    df['MA_20'] = df['Close'].rolling(20).mean()
    
    # Drop rows with NaN from rolling calculations
    df.dropna(inplace=True)
    
    return df

In [30]:
# Importing Data
print(df.head())
print(df.columns)

Price           Close       High        Low       Open     Volume
Ticker           AAPL       AAPL       AAPL       AAPL       AAPL
Date                                                             
2015-01-02  24.261047  24.729270  23.821672  24.718174  212818400
2015-01-05  23.577578  24.110154  23.391177  24.030267  257142000
2015-01-06  23.579798  23.839428  23.218089  23.641931  263188400
2015-01-07  23.910439  24.010296  23.677436  23.788389  160423600
2015-01-08  24.829126  24.886823  24.121244  24.238856  237458000
MultiIndex([( 'Close', 'AAPL'),
            (  'High', 'AAPL'),
            (   'Low', 'AAPL'),
            (  'Open', 'AAPL'),
            ('Volume', 'AAPL')],
           names=['Price', 'Ticker'])


In [29]:
# Test
# Printing first 5 rows of cleaned data
df_cleaned = clean_stock_data(df)
print(df_cleaned.head())

         Date      Close       High        Low       Open     Volume  \
19 2015-01-30  25.998575  26.628789  25.929783  26.273739  334982000   
20 2015-02-02  26.324781  26.444611  25.758921  26.196077  250956400   
21 2015-02-03  26.329216  26.426854  26.098433  26.295930  207662800   
22 2015-02-04  26.531153  26.741965  26.253769  26.295932  280598800   
23 2015-02-05  26.720516  26.785123  26.566796  26.738338  168984800   

    Daily_Change       MA_5      MA_20  
19     -1.463416  25.457568  24.552078  
20      1.254708  25.702997  24.655265  
21      0.016846  26.125063  24.792847  
22      0.766968  26.313684  24.940415  
23      0.713740  26.380848  25.080918  


## Part 3:
(2\%) Builds at least two visualizations (graphs/plots) from the data which help to understand or answer the questions of interest. These visualizations will be graded based on how much information they can effectively communicate to readers. Please make sure your visualization are sufficiently distinct from each other.

In [31]:
# importing necessary libraries
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import altair as alt


# Only plots need to be made


1) Time Series Plot of Closing Price with Moving Averages

In [37]:
import altair as alt

chart_close = alt.Chart(df_cleaned).mark_line(color='blue').encode(
    x='Date:T',
    y='Close:Q',
    tooltip=['Date:T', 'Close:Q']
).properties(
    width=800,
    height=400,
    title='Apple Closing Price Over Time'
).interactive()  # Enables zooming and panning

chart_close


alt.Chart(...)

2) Scatter Plot: Volume vs Daily Percent Change


In [38]:
chart_scatter = alt.Chart(df_cleaned).mark_circle(size=60, opacity=0.5).encode(
    x='Volume:Q',
    y='Daily_Change:Q',
    color=alt.condition("datum.Daily_Change>0", alt.value('green'), alt.value('red')),
    tooltip=['Date:T', 'Daily_Change:Q', 'Volume:Q']
).properties(
    width=800,
    height=400,
    title='Daily Percent Change vs Trading Volume'
).interactive()  # Zoom/pan

chart_scatter


alt.Chart(...)

3) Histogram of Daily Percent Change

In [39]:
chart_hist = alt.Chart(df_cleaned).mark_bar().encode(
    x=alt.X('Daily_Change:Q', bin=alt.Bin(maxbins=50)),
    y='count()',
    tooltip=['count()']
).properties(
    width=800,
    height=400,
    title='Distribution of Daily Percent Change'
).interactive()  # Zoom on x-axis

chart_hist


alt.Chart(...)